# Basic Graph Neural Networks with Python

> Philipp Zimmermann

### Imports

In [1]:
# Logic
import torch
from torch_geometric.data import Data

# Visualization
from IPython.display import clear_output
import ipywidgets as widgets

OSError: libcudart.so.10.2: cannot open shared object file: No such file or directory

### Versions

In [2]:
# PyTorch
TORCH = torch.__version__
print(f'PyTorch: {TORCH}')

# Cuda
CUDA = torch.version.cuda
print(f'Cuda:    {CUDA}')

PyTorch: 1.8.0+cu111
Cuda:    11.1


### Graph Object

In [ ]:
# 3 nodes
# each with one feature: -1, 0 and 1
nodes = torch.tensor([[-1], [0], [1]], dtype=torch.float)

# 4 edges 
# 
# edge_index represents their source and target node
# undirected graph, since both edges go in both directions
# unweighted because edges do not have values
edge_index = torch.tensor([[0, 1], 
                           [1, 2], 
                           [1, 0], 
                           [2, 1]], dtype=torch.long)

# Graph
graph = Data(x=nodes, edge_index=edge_index.t().contiguous())

#### Information of Graph

In [ ]:
# commands
cmds = [
    ['Graph Object',       'graph',                           graph],
    ['Amount of nodes',    'graph.num_nodes',                 graph.num_nodes],
    ['Amount of edges',    'graph.num_edges',                 graph.num_edges],
    ['Amount of features', 'graph.num_node_features',         graph.num_node_features],
    ['Isolated nodes',     'graph.contains_isolated_nodes()', graph.contains_isolated_nodes()],
    ['Self loops',         'graph.contains_self_loops()',     graph.contains_self_loops()],
    ['Directed',           'graph.is_directed()',             graph.is_directed()]
]

print('  Description\t\tCommand\t\t\t\t\tOutput\n')
print('-' * 99)
for description, cmd, output in cmds:
    print(f'  {description}{" " * (20 - len(description))}\t{cmd}{" " * (34 - len(cmd))}\t{output}')

#### Use Cuda

In [ ]:
# transfer data object to GPU
device = torch.device('cuda')
graph = graph.to(device)

### Data Handling

In [ ]:
# Dataset
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [ ]:
# information about the dataset
print(f'Length:   {len(dataset)}\nClasses:  {dataset.num_classes}\nFeatures: {dataset.num_node_features}')

In [ ]:
# shuffle the dataset
dataset = dataset.shuffle()

In [ ]:
# training and testing dataset
train_data = dataset[:540]
test_data  = dataset[540:]

In [ ]:
# Batches
from torch_geometric.data import DataLoader

loader = DataLoader(dataset, batch_size=64, shuffle=True)

for data in loader:
    print(data)

### Learning Methods

In [ ]:
# dataset
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
# imports
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [ ]:
# 2-layer GCN
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [ ]:
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model     = Net().to(device)
data      = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

### References

[1] https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html